<a href="https://colab.research.google.com/github/shizoda/education/blob/main/agent/LangChain(3)_StructuredOutput_ToolCalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 第3回：構造化出力とツール利用

LangGraph などのエージェントシステムを作る際には、LLM と外部プログラムの接続が重要です。今回は、そのための2つの技術を学びます。

1. **Structured Output (構造化出力):** LLM のふわっとした言葉を、システムで使える「確定したデータ型」に変換する。
2. **Tool Calling (ツール利用):** LLM が自ら「計算したい」「検索したい」と判断し、Python 関数を呼び出せるようにする。

### 🎯 今回の学習目標
* 第1〜2回の知識（Prompt, Chain）が定着しているか確認する。
* **Pydantic** を使い、LLM から JSON データを抽出する方法を学ぶ。
* **`@tool`** デコレータと **`.bind_tools()`** を使い、LLM に Python 関数を認識させる。

### ⚡ 環境構築とAPIキー設定

いつものセットアップを行います。
必要なライブラリに加え、今回はデータ定義のために `pydantic` も確認しますが、これはColabには標準で入っています。

In [ ]:
!pip install -qU langchain-openai langchain-core termcolor pydantic

import os
from google.colab import userdata
from termcolor import cprint

try:
    if "OPENROUTER_API_KEY" not in os.environ:
        os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")
    cprint("✅ API Key loaded.", "green")
except Exception as e:
    cprint("❌ Error: Please set OPENROUTER_API_KEY in secrets.", "red")

## 1️⃣ 【復習】 基礎知識チェック（穴埋め課題）

LangChain の基礎となる「プロンプト」と「LCEL」の理解度を確認しましょう。
以下のコードの `___` の部分を書き換えて、正常に動くコードを完成させてください。

**課題の要件:**
1. `ChatPromptTemplate` を使って、「料理名 {dish} のレシピを教えて」というテンプレートを作る。
2. LCEL のパイプ `|` を使って、 `prompt` → `llm` → `parser` を繋ぐ。
3. `invoke` で実行する。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# モデルの準備
llm = ChatOpenAI(
    model="deepseek/deepseek-chat-v3-0324", # または "openai/gpt-4o-mini"
    api_key=os.environ["OPENROUTER_API_KEY"],
    base_url="https://openrouter.ai/api/v1",
    temperature=0.5
)

# --- 📝 ここから修正してください ---

# 1. プロンプトテンプレートの作成
# ヒント: 文字列からテンプレートを作るメソッドは .from_template(...)
prompt = ChatPromptTemplate.___( "{dish} の美味しい作り方を一言で教えて。" )

# 2. 文字列出力パーサーの準備
parser = StrOutputParser()

# 3. チェーンの構築 (LCEL)
# ヒント: パイプ演算子 | で繋ぎます
chain = prompt ___ llm ___ parser

# 4. 実行
# ヒント: 実行メソッドは .invoke(...)
result = chain.___(
    {"dish": "カレーライス"}
)

# --- 修正ここまで ---

print(f"▼ 結果:\n{result}")

## 2️⃣ Structured Output（構造化出力）

### なぜ必要なのか？
通常のチャット (`StrOutputParser`) では、LLM は「文字列」を返します。しかし、システム開発では以下のような状況が頻発します。

* 「ユーザーのプロフィール文から、 **名前** と **年齢** と **趣味** をデータベースに保存したい」
* 「長文のニュースから、 **重要キーワード** をリスト形式で取り出したい」

これを「正規表現」や「文字列操作」で行うのはバグの元です。
LangChain では **Pydantic** (Python のデータ定義ライブラリ) と **`.with_structured_output()`** を使うことで、LLM に強制的に指定した型で出力させることができます。

### 実践：ユーザー情報の抽出
以下の自己紹介文から、データを取り出してみましょう。

> 「こんにちは！私はケンです。30歳で、東京でエンジニアをしています。休日はサウナに行くのが好きです。」

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional, List

# 1. 設計図 (スキーマ) を作る
# Pydanticを使って「どんなデータが欲しいか」をクラスとして定義します。
class UserProfile(BaseModel):
    name: str = Field(description="ユーザーの名前")
    age: Optional[int] = Field(description="ユーザーの年齢。不明な場合はNone")
    job: str = Field(description="職業")
    hobbies: List[str] = Field(description="趣味のリスト")

# 2. モデルに構造化機能を付与する
# .with_structured_output(クラス名) を使うと、出力がそのクラスのインスタンスになります。
structured_llm = llm.with_structured_output(UserProfile)

# 3. 実行
text_input = "こんにちは！私はケンです。30歳で、東京でエンジニアをしています。休日はサウナに行くのが好きです。"
profile = structured_llm.invoke(text_input)

# 結果の確認
print(f"▼ 型の確認: {type(profile)}")
print(f"▼ 名前: {profile.name}")
print(f"▼ 年齢: {profile.age}")
print(f"▼ 趣味: {profile.hobbies}")

# JSON形式 (辞書) に変換するのも簡単です
print(f"\n▼ JSON形式:\n{profile.model_dump()}")

### 💡 ポイント
もし入力テキストに「年齢」が含まれていなかったらどうなるでしょうか？
上記の `text_input` を変更して（例：「私はケンです。」だけにする）、 `age` がどうなるか試してみてください。

`Optional[int]` と定義し、 `Field` に説明を書いているため、LLM は「情報がない場合は None にする」という判断を自動で行います。

### 📝 【演習】 Pydantic による注文情報の抽出

自己紹介文以外のパターンでも練習してみましょう。
次は **「ECサイトのチャットボット」** を想定します。
ユーザーは自然言語で注文を行いますが、システム側では「商品名」や「数量」などの確定したデータが必要です。

以下の要件を満たす `Order` クラスを定義し、注文から:データを抽出してください。

**抽出したいデータ（スキーマ）:**
* `product_name` (str): 商品名。
* `quantity` (int): 注文数量。
* `express_delivery` (bool): お急ぎ便の希望有無。文脈から判断させます（デフォルトは False）。
* `price_limit` (Optional[int]): 予算上限。指定がない場合は None。

この演習を通して、 **「ライブラリのインポートから実行結果の取り出しまで」** の一連の流れを構築する力を養います。

In [ ]:
# --- 📝 以下のコードの ___ を書き換えて完成させてください ---

# 1. 必要なライブラリのインポート
# ヒント: データ定義には BaseModel, Field が必要です
# ヒント: 予算上限など「値がない場合」を扱うために Optional が必要です
from pydantic import ___, ___
from typing import ___

# 2. データ構造 (スキーマ) の定義
# ヒント: Pydanticのモデルは BaseModel を継承します
class Order(___):
    # 商品名: 文字列型。Fieldで説明を追加 ("注文する商品の名前")
    product_name: str = ___

    # 数量: 整数型
    quantity: int = Field(description="注文する数量")

    # お急ぎ便: 真偽値型 ("お急ぎ便や速達を希望している場合はTrue、そうでなければFalse")
    express_delivery: bool = ___

    # 予算上限: 整数型だが、指定がない場合は None (Optional[int])
    price_limit: ___ = Field(description="予算の上限金額。指定がなければNone")

# 3. LLMにスキーマを適用
# ヒント: .with_structured_output() メソッドを使います
order_llm = llm.___(Order)

# 4. テストデータの実行
user_order = "来週のキャンプ用に、テントを2つ買いたいです。なるべく早く届けてほしい！予算はだいたい3万円以内で。"

# ヒント: invoke メソッドで実行します
result = order_llm.___(user_order)

# 5. 結果の表示
# ヒント: 結果は Order クラスのインスタンスとして返ってきます。属性 (.product_name など) でアクセスします
print(f"▼ 注文商品: {result.___}")
print(f"▼ 数量: {result.___} 個")
print(f"▼ お急ぎ便: {result.___}")
print(f"▼ 予算上限: {result.___} 円")

# --- 修正ここまで ---

## 3️⃣ Tool Calling（ツール利用）

### ツール利用とは
LLM は計算が苦手ですし、インターネット検索も単体ではできません。
LLM に Python の関数（ツール）を渡し、「必要ならこれを使ってください」と教える機能が **Tool Calling** です。

LangGraph のエージェントは、この仕組みを使って、自分で考え、ツールを選び、実行するようなサイクルを回しています。

### 例：計算ツール
LLM は複雑な計算を間違えることがありますが、Python の関数を使えば正確です。

In [ ]:
from langchain_core.tools import tool

# 1. ツールの定義
# ＠tool デコレータをつけ、docstring（関数の説明）をしっかり書くのがコツです。
# LLM はこの説明文を読んで「いつ使うべきか」を判断します。

@tool
def multiply(a: int, b: int) -> int:
    """2つの整数を掛け算します。"""
    return a * b

# 確認：ツールには名前や説明が自動的に付与されます
print(f"Tool Name: {multiply.name}")
print(f"Tool Description: {multiply.description}")

# 2. モデルにツールをバインド (紐づけ) する
# bind_tools でリスト形式で渡します
llm_with_tools = llm.bind_tools([multiply])

# 3. 実行：ツールが必要な質問を投げる
query = "123 かける 456 はいくつ？"
ai_msg = llm_with_tools.invoke(query)

# 重要：ここでの戻り値 ai_msg は「計算結果」ではありません！
# 「計算関数を、引数(123, 456)で呼び出したい」という「リクエスト」が入っています。
print(f"\n▼ AIの返答 (Message):\n{ai_msg}")
print(f"\n▼ ツール呼び出しリクエスト (tool_calls):\n{ai_msg.tool_calls}")

### 📝 【追加演習 1】 Tool Calling による現在時刻の取得

天気予報の代わりに、**「現在時刻」** を取得させてみましょう。
LLM は学習した時点の知識で止まっているため、「今、何時？」と聞かれても答えることができません（あるいは適当な嘘をつきます）。

そこで、Python の `datetime` モジュールを使って **「本当に現在の時刻を返す関数」** を作成し、それをツールとして LLM に持たせます。これならダミーデータではなく、実行するたびに結果が変わるリアルなシステムになります。

**課題:**
1. `@tool` デコレータを使って、時刻取得関数をツール化する。
2. `bind_tools` で LLM にツールを認識させる。
3. 「今の東京の時間は？」と質問し、LLM がツールを使おうとしたか確認する。

In [ ]:
# --- 📝 以下のコードの ___ を書き換えて完成させてください ---

# 1. 必要なライブラリのインポート
from langchain_core.tools import ___
from datetime import datetime
from zoneinfo import ZoneInfo

# 2. ツールの定義
# ヒント: デコレータを使って関数をツール化します
# ヒント: docstring ("""...""") はAIが使い方を判断するために重要です
@___
def get_current_time(timezone: str = "Asia/Tokyo") -> str:
    """
    指定されたタイムゾーンの現在時刻を取得します。
    引数 timezone には 'Asia/Tokyo' や 'UTC' などの文字列を指定します。
    """
    # 実際に現在時刻を取得するコード（ダミーではありません！）
    now = datetime.now(ZoneInfo(timezone))
    return now.strftime("%Y-%m-%d %H:%M:%S")

# 3. モデルにツールをバインド (紐づけ)
# ヒント: 定義した関数をリスト形式で渡します
time_llm = llm.___( [___] )

# 4. 実行
# わざと少し複雑な聞き方をして、引数 (Asia/Tokyo) を推論させてみます
query = "今、東京は何時ですか？"

# ヒント: invoke で実行します
ai_msg = time_llm.___(query)

# 5. 結果の確認
# ヒント: ツール呼び出しの情報は .tool_calls プロパティに入っています
print(f"▼ AIの返答オブジェクト:\n{ai_msg}")
print(f"\n▼ ツール呼び出しリクエスト:\n{ai_msg.___}")

# --- 修正ここまで ---

# (参考) 正しく実装できていれば、tool_calls の中に
# {'name': 'get_current_time', 'args': {'timezone': 'Asia/Tokyo'}, ...}
# というデータが入っているはずです。

### 📝 【追加演習 2】 世界の都市の時刻推論

LLM にツールを使わせる最大のメリットは、**「曖昧なユーザーの言葉を、正確な引数に変換してくれること」** です。

先ほど定義した `get_current_time` 関数は、`Asia/Tokyo` や `America/New_York` といった正確な **Time Zone ID** を引数として受け取る必要があります。しかし、ユーザーはいちいちそんな ID を覚えていません。「ロンドン」や「NY」と入力するだけです。

LLM が文脈を理解し、適切な ID を推論してツールに渡してくれることを確認するために、複数の都市について連続で問い合わせてみましょう。

**課題:**
1. 調査したい都市のリストを作成する（ロンドン、ニューヨーク、北京など）。
2. `for` ループを使って、各都市の時刻を問い合わせる。
3. LLM が生成したツール呼び出し引数 (`args`) が、正しいタイムゾーンIDになっているか確認する。

In [ ]:
# --- 📝 以下のコードの ___ を書き換えて完成させてください ---

# 1. 調査したい都市や国名のリスト
# ヒント: "ロンドン", "New York" のほかに "北京" や "Beijing"、"ペキン" なども試してみましょう
target_cities = ["ロンドン", "New York", "___"]

print(f"--- 🌍 世界時計ツアー開始 ---")

for city in target_cities:
    # 質問文を作成
    query = f"{city}の現在の時刻を教えてください。"

    # 2. LLMを実行
    # ヒント: 先ほど作成した time_llm を再利用します
    ai_msg = time_llm.___(query)

    # 3. 推論結果の確認
    if ai_msg.tool_calls:
        # ツール呼び出しの引数を取り出す
        args = ai_msg.tool_calls[0]["args"]
        timezone_id = args.get("timezone")

        # 結果表示
        print(f"\n都市名: {city}")
        print(f" -> 🤖 推論されたタイムゾーン: {timezone_id}")

        # 実際にツールを動かして時刻を表示してみる（オプション）
        # time_tool = get_current_time (関数の実体)
        # print(f" -> ⏰ 実際の時刻: {get_current_time.invoke(args)}")
    else:
        print(f"\n都市名: {city} -> ⚠️ ツールが呼ばれませんでした")

# --- 修正ここまで ---

# (解説)
# "北京", "Beijing", "ペキン" のいずれを入力しても、
# LLM は 'Asia/Shanghai' という正しいタイムゾーンIDを推論するはずです。
# これが「LLMをインターフェースにする」利点です。

### 🔬 【補足】 Tool Calling における LLM の役割

ここで重要な点は、`llm.bind_tools` を使ったものの、LLM が自分で関数を実行してくれたわけではないことです。

LLM が行ったのは、あくまで **「この関数を、この引数で実行してください」という指示文（`tool_calls`）を作成しただけ** です。この指示文を受け取った **LangChain 側** が、実際にその関数を実行します。

In [ ]:
# 直前の実行結果 (ai_msg) から、ツール呼び出しの「中身」を取り出します
if ai_msg.tool_calls:
    # 1. LLMが作成したリクエストを取得 (リストの1つ目)
    request = ai_msg.tool_calls[0]

    # 2. 関数名と引数を確認
    func_name = request["name"]
    args = request["args"]

    print(f"🤖 LLMの指示: 「関数 {func_name} を 引数 {args} で実行せよ」")

    # 3. 人間（システム）が代わりに関数を実行
    # ここでは手動で行いますが、エージェントシステムではここを自動化します
    if func_name == "get_current_time":
        # 定義しておいた関数に引数を渡して実行
        tool_result = get_current_time.invoke(args)
        print(f"✅ 実行結果: {tool_result}")

else:
    print("⚠️ ツール呼び出しのリクエストがありませんでした")

## 🏆 第1回〜第3回のまとめ：LangChain 基礎コース修了

これまでの3回で、LangChain の重要な知識をすべてカバーしました。

### 🎓 まとめ

| 回 | テーマ | キーワード | 学んだこと |
|:---:|:---|:---|:---|
| **第1回** | **LLMの基礎** | `ChatModel`, `SystemMessage`, `invoke` | API を阿叩く上での **役割(Role)** を持たせた会話の基本構造を学びました。 |
| **第2回** | **プロンプトとLCEL** | `PromptTemplate`, `OutputParser`, `\|` (Pipe) | 入力から出力までを **パイプライン** としてつなぎ、効率的に処理する方法を学びました。 |
| **第3回** | **システムとの接続** | `Structured Output`, `Tool Calling` | LLMをただのチャットボットではなく、外部のツールを呼ぶためのツールとして使うようになりました。 |

---

### 🚀 次は LangGraph へ

これまでは、一本道の処理（チェーン）を作ってきました。
しかし、現実の複雑なタスク（例：「検索してみて、見つからなかったら別の方法を試す」など）には、**ループ（繰り返し）** や **条件分岐** が必要です。次回からは **LangGraph** を使い、具体的な AI エージェントの構築に入ります。